#References
For inital word embeddings (BERT): https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

Encoder: BERT

Encoder-Decoder: T5

In [ ]:
!pip install transformers
!pip install transformers datasets evaluate scikit-learn
!pip install [transformers](https://saturncloud.io/glossary/transformers) #Hugging Face transformers library
!pip install sentencepiece
! pip install -U accelerate
! pip install -U transformers
!pip install transformers[torch]

#Load Bert Encoder Model

In [ ]:
import torch

if(torch.cuda.is_available()):
    device = "cuda:0"
else:
    device = "cpu"
print(f"GPU or CPU? - {device}")

GPU or CPU? - cuda:0


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from tqdm import tqdm

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

#Load in Dataset

In [ ]:
import pandas as pd
import re
from google.colab import drive

drive.mount('/gdrive')

csv = "/gdrive/My Drive/Colab Notebooks/CSCI5541/DataCreation/Sorted_Cleaned_Articles.csv"
df = pd.read_csv(csv)

Mounted at /gdrive


In [ ]:
df['wiki_label'][0]

' "Rivaroxaban in Patients with Recent Acute Coronary Syndrome".The New England Journal of Medicine. 2012. 366(1):9-19.PubMed•Full text•PDFContents1Clinical Question2Bottom Line3Major Points4Guidelines5Design6Population6.1Inclusion Criteria6.2Exclusion Criteria6.3Baseline Characteristics6.4Baseline Medications7Interventions8Outcomes8.1Primary Outcomes8.2Secondary Outcome8.3Additional Analyses8.4Adverse Events8.5Subgroup Analysis9Criticisms10Funding11Further ReadingClinical QuestionIn patients with recent ACS, does the addition of rivaroxaban to standard dual antiplatelet therapy improve CV morbidity and mortality?Bottom LineIn patients with recent ACS, the addition of rivaroxaban to standard dual antiplatelet therapy reduced the composite of CV mortality, recurrent MI, or stroke, but increased the risk of nonfatal bleeding.Major PointsTwo meta-analyses of RCTs investigating the addition of warfarin to aspirin among ACS patients suggested that combination therapy may reduce the risk of 

#Tokenize our Dataset

##Trying to map data - truncation will not work within preprocess function as some entries are empty strings.

In [ ]:
#!pip install datasets

from datasets.io.abc import DatasetDict
Dataset_cleaned = DatasetDict({
    'methods' : df['methods'],
    'disussion' : df['disussion'],
    'results' : df['results']})



ModuleNotFoundError: ignored

In [ ]:
#Dataset_cleaned['methods']

#dd = DatasetDict({"methods":df['methods'],"discussion":df['disussion'], "results":df['results']})

methods = df['methods']

In [ ]:
methods

0      Methods STUDY POPULATION The study included pa...
1      Methods The study protocol has been described ...
2      Methods STUDY OVERSIGHT The design of the CHAM...
3      Methods TRIAL DESIGN The Clopidogrel for High ...
4      Methods TRIAL DESIGN AND OVERSIGHT In this ran...
                             ...                        
326    Methods STUDY DESIGN Details of the study desi...
327    Methods STUDY DESIGN From February 2009 throug...
328    Methods STUDY DESIGN AND OVERSIGHT The INPULSI...
329    Methods STUDY SITES AND PATIENTS The study was...
330    Methods STUDY DESIGN AND PATIENTS The first an...
Name: methods, Length: 331, dtype: object

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["methods"], truncation=False)

tokenized_data = Dataset_cleaned.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

##Example of how we are tokenizing.

In [ ]:
# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(df['abstract_background'][0])

# Print out the tokens.
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

vocab = {v: k for k, v in tokenizer.vocab.items()}
print([vocab[id] for id in indexed_tokens])

['background', 'acute', 'corona', '##ry', 'syndrome', '##s', 'arise', 'from', 'corona', '##ry', 'at', '##her', '##os', '##cle', '##rosis', 'with', 'super', '##im', '##posed', 'th', '##rom', '##bos', '##is', '.', 'since', 'factor', 'x', '##a', 'plays', 'a', 'central', 'role', 'in', 'th', '##rom', '##bos', '##is', ',', 'the', 'inhibition', 'of', 'factor', 'x', '##a', 'with', 'low', '-', 'dose', 'ri', '##var', '##ox', '##aba', '##n', 'might', 'improve', 'cardiovascular', 'outcomes', 'in', 'patients', 'with', 'a', 'recent', 'acute', 'corona', '##ry', 'syndrome', '.', 'methods']
[4281, 11325, 21887, 2854, 8715, 2015, 13368, 2013, 21887, 2854, 2012, 5886, 2891, 14321, 29166, 2007, 3565, 5714, 19155, 16215, 21716, 15853, 2483, 1012, 2144, 5387, 1060, 2050, 3248, 1037, 2430, 2535, 1999, 16215, 21716, 15853, 2483, 1010, 1996, 23586, 1997, 5387, 1060, 2050, 2007, 2659, 1011, 13004, 15544, 10755, 11636, 19736, 2078, 2453, 5335, 22935, 13105, 1999, 5022, 2007, 1037, 3522, 11325, 21887, 2854, 8715,

##Set Model to evaluate. Process token representation of component.

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

model = model.to(device)
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

NLTK package resulted in a few sentences having over +3,000 tokens. Thanks to https://stackoverflow.com/questions/4576077/how-can-i-split-a-text-into-sentences#4576110 we were able to find a package similar to NLTK's sent_tokenize but with the allowance of added flexibility in adjusting how sentences are split.

We add the semicolon character as a means for splitting a sentence as token count was too large for inputting into BERT for a few sentences.

In [ ]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace(";",";<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences


##Create Encoding Dataset

In [ ]:
import nltk
nltk.download('punkt')
data_list = [df['methods'], df['results'], df['disussion']]
data_finalized = {}

for data_index, data_type in enumerate(data_list):
  print(data_index + 1, " of ", len(data_list))
  data_finalized[data_index] = []
  for index, data in enumerate(tqdm(data_type)):
    #Tokenize our sentence with the BERT tokenizer.
    try:
      sentences = split_into_sentences(data)
    except:
      sentences = ["NAN"]

    for index_sentence, sentence in enumerate(sentences):
      #Add special tokens
      sentence = "[CLS] " + sentence + " [SEP]"
      tokenized_text = tokenizer.tokenize(sentence)

      # Map the token strings to their vocabulary indeces.
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

      segments_ids = [1] * len(tokenized_text)
      #print(segments_ids)

      # Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])

      #Place model inputs on GPU
      tokens_tensor = tokens_tensor.to(device)
      segments_tensors = segments_tensors.to(device)

      # Run the text through BERT, and collect all of the hidden states produced
      # from all 12 layers.
      with torch.no_grad():
        # Evaluating the model will return a different number of objects based on
        # how it's  configured in the `from_pretrained` call earlier. In this case,
        # becase we set `output_hidden_states = True`, the third item will be the
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

        if(index_sentence != 0):
          hidden_states += hidden_states

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # `hidden_states` has shape [13 x 1 x 68 x 768]

#   `token_vecs` is a tensor with shape [68 x 768]
    token_vecs = hidden_states[-2][0]

    #Create our embedding vector
    sentence_embedding = torch.mean(token_vecs, dim=0)
    data_finalized[data_index].append(sentence_embedding)

##Transfer our dictionary of outputs from GPU back to CPU. Once transferred convert to pandas dataframe then write to Google Drive.

In [ ]:
len(data_finalized)
dictionary = {}

for list_value in data_finalized:
  #print(list_value)
  dictionary[list_value] = []
  for value in data_finalized[list_value]:
    #print(len(value))
    dictionary[list_value].append(value.cpu().numpy())


In [ ]:
final_list = []
for value in range(len(data_finalized[0])):
    input = torch.cat((data_finalized[0][value], data_finalized[1][value], data_finalized[2][value]), 0)
    final_list.append(input)

In [ ]:
type(df['wiki_label'])

labels = df['wiki_label'].to_list()
indexes = df['Unnamed: 0'].to_list()

In [ ]:
from datasets.io.abc import DatasetDict
from datasets import Dataset

Dataset_cleaned = DatasetDict({
    'train' : Dataset.from_dict({
        'idx' : indexes,
        'input' :  final_list,
        'labels' : labels
    })
})

def preprocess_function(examples):
    return tokenizer(examples, truncation=True)

tokenized_data = Dataset_cleaned.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
Dataset_cleaned

DatasetDict({
    train: Dataset({
        features: ['idx', 'input', 'labels'],
        num_rows: 331
    })
})

In [ ]:
#output_df.to_csv("/gdrive/My Drive/Colab Notebooks/CSCI5541/DataCreation/Auto_Encoder_DataSet.csv")

#PEGASUS-XSUM Model

References: https://huggingface.co/google/pegasus-xsum

Encoder-Decoder: Pegasus-XSUM

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
import math
import time
from transformers import AdamW
from transformers import get_scheduler



drive.mount('/gdrive')
save_path = '/gdrive/My Drive/Colab Notebooks/CSCI5541/Model/parameter_config.pth'

writer_loss = SummaryWriter()
writer_acc = SummaryWriter()

def train():
  EPOCHS = 10
  LEARNING_RATE = 0.001
  train_loss =[]
  train_acc =[]
  eval_acc =[]
  eval_loss = []
  batch_loss = []



  criterion = torch.nn.CrossEntropyLoss().to(device)
  optimizer = torch.optim.AdamW(model.parameters() , lr=LEARNING_RATE)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1 , gamma=0.9)

  max_steps = math.ceil(EPOCHS * len(data_finalized[0]))

  #Optimizer
  optimizer = AdamW(model.parameters(), lr=5e-5)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer = optimizer,
      num_warmup_steps = 0,
      num_training_steps = max_steps
  )

  for epoch in range(EPOCHS):
    train_loss_per_epoch = 0
    train_acc_per_epoch = 0

    eval_loss_per_epoch = 0
    eval_acc_per_epoch = 0
    for data_index in tqdm(range(len(data_finalized[0]))):
      #Tokenize Data)
      results = data_finalized[0][data_index]
      methods = data_finalized[1][data_index]
      discussion = data_finalized[2][data_index]

      inputs = torch.cat((results, methods, discussion), 0).to(device)
      #print(len(inputs))
      inputs = str(inputs)

      input = tokenizer.prepare_seq2seq_batch([inputs], max_length=16000, truncation = True, return_tensors='pt')
      input = input.to(device)

      output = tokenizer(df['wiki_label'][data_index], max_length = 16000, truncation = True, return_tensors = 'pt')

      # forward pass
      summary_ids = model.generate(**input)

      summary = tokenizer.batch_decode(summary_ids, skip_special_tokens = True, return_tensors = 'pt')
      print(len(summary[0]))


      print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))

      #Computing the loss
      #loss = criterion(summary, output['input_ids'])
      #loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

train()


In [ ]:
data_finalized['wiki_label']